In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd
import csv

In [ ]:
# Provided parameter values
# a = np.array([1.44459561, 1.4909716, 1.29900401, 1.59823275, 1.95377401,
#               1.29692538, 1.3884494, 1.41366705, 1.4720679, 1.94549286,
#               1.42256119, 1.35419586, 1.0655931, 1.4807465, 3.57915218,
#               1.28707514, 1.29840168, 1.42614346, 2.301248, 1.82484445,
#               1.37406175, 1.32049406, 1.48991211, 2.36839102, 2.57548982])
# OD = np.array([10.62209668, 5.29927502, 5.24461367, 3.95714724, 3.53289089,
#                4.90284341, 4.77882344, 4.21487304, 2.88143791, 3.68379522,
#                7.11055405, 6.37264251, 5.52053175, 4.6822641, 3.91513311,
#                3.79516812, 16.75179172, 4.82518489, 3.90599822, 4.43170833,
#                4.68538009, 5.17359904, 5.77046094, 4.69307385, 4.32273257])
# g = np.array([1018.98154797, 2459.73045748, 1571.1443399, 4717.39214842,
#               7757.19215507, 1678.80384039, 2371.79623817, 2763.04246768,
#               5398.06831678, 6639.08957198, 1498.46771748, 1437.98348092,
#               896.57655106, 2596.58011685, 15084.55697028, 2336.91028423,
#               456.33896778, 2464.47850684, 8199.06494407, 4530.89552788,
#               2122.01729478, 1659.39463412, 2229.42490177, 6774.64313489,
#               8393.47117689])


# Reading the parameters from the csv file
csv_file_path = r"C:\Users\zxq220007\Box\Quantum Optics Lab\TeTON OANN Testbed\Data 2024\Apr 16 2024\5X5 Trans5 50mm cell 130C 290MHz 3037MHz\Results 2\Optimized_Curve_Fitting_Parameters.csv"

# Indicating the concavity of the non linear funtion used for curve fitting
# 1: Concave up exponential function
# 2: Concave down exponential function
function_tpye = 1

# Initializing the arrays to store the parameters
a = np.array([])
OD = np.array([])
g = np.array([])

# Reading the csv file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip header row

    for row in csv_reader:
        a = np.append(a, float(row[1]))
        OD = np.append(OD, float(row[2]))
        g = np.append(g, float(row[3]))

In [ ]:
# Define the custom activation function
class CustomNonlinearLayer1(nn.Module):
    def __init__(self, a, OD, g):
        super(CustomNonlinearLayer1, self).__init__()
        self.a = nn.Parameter(torch.tensor(a, dtype=torch.float32), requires_grad=False)
        self.OD = nn.Parameter(torch.tensor(OD, dtype=torch.float32), requires_grad=False)
        self.g = nn.Parameter(torch.tensor(g, dtype=torch.float32), requires_grad=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Non-linear function to each neuron based on a, OD, and g
        x = self.a * torch.exp(-self.OD * self.g / (x * 12800 + self.g))
        return x

class CustomLinearLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomLinearLayer, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))  # Weight matrix only, no bias
        nn.init.xavier_uniform_(self.weight)  # Initialize the weight matrix

    def forward(self, x):
        return torch.matmul(x, self.weight.t())  # Matrix multiplication

# Define the model
class Net(nn.Module):
    def __init__(self, a, OD, g):
        super(Net, self).__init__()
        self.fc1 = CustomLinearLayer(144, 25)
        self.nonlinear = CustomNonlinearLayer1(a, OD, g)
        self.fc2 = CustomLinearLayer(25, 10)

    def forward(self, x):
        x = x.view(-1, 144)
        x = self.fc1(x)
        x = self.nonlinear(x)
        x = self.fc2(x)
        return x

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model and move it to the device
a = 0.5  # Example values, replace with appropriate values
OD = 0.1  # Example values, replace with appropriate values
g = 0.2  # Example values, replace with appropriate values
model = Net(a, OD, g).to(device)

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.Resize((12, 12)),  # Resize the images to 12x12 pixels
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)


# Training loop
for epoch in range(10):  # Number of epochs
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 12*12).to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Testing the model after each epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 12*12).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Epoch [%d/%d], Accuracy of the network on the test images: %.2f %%' % (epoch+1, 10, 100 * correct / total))